In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh
import geopandas as gpd
import datetime

# pd.set_option('plotting.backend', 'pandas_bokeh')
pandas_bokeh.output_notebook()


Loading BokehJS ...

# Svetová mapa

In [18]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv", parse_dates=["date"]
).loc[
    lambda df: ~df["iso_code"].str.contains("_"),
    ["iso_code", "location","date", "total_cases","total_cases_per_million", "total_deaths", "total_deaths_per_million", "new_cases"],
]

grouped_data = (df.sort_values('date')
                    .groupby('iso_code')
                    .tail(1)
                    .drop(columns=["new_cases"]))


In [19]:
grouped_data = grouped_data.set_index('iso_code').join(
    (df[df.date > datetime.datetime.now() - pd.to_timedelta("30day")] \
    .groupby('iso_code')
    .sum('new_cases')
    .rename(columns={'new_cases': 'cases_last_30d'}))['cases_last_30d'])


In [20]:
gdf = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")).loc[:, ["iso_a3", "name", "geometry"]]
gdf.loc[gdf.name == 'France', 'iso_a3'] = 'FRA'    
gdf.loc[gdf.name == 'Norway', 'iso_a3'] = 'NOR'    

gdf = gdf.merge(grouped_data, left_on="iso_a3", right_on="iso_code")
gdf['date'] = gdf['date'].dt.strftime('%Y-%m-%d')


In [21]:
gdf.plot_bokeh(
        dropdown=["total_deaths", "total_cases", "cases_last_30d","total_cases_per_million","total_deaths_per_million"],
        show_colorbar=False,
        colormap="RdYlGn",
        hovertool_string="<h3>@name</h3><pre>@Colormap{0.0a}</pre>",
        figsize=(1200, 600),
    )

/home/egruy/.local/lib/python3.8/site-packages/pandas_bokeh/geoplot.py:118: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:


Column(id='4733', ...)

# Porovnanie krajín

In [57]:
data = (pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv", parse_dates=['date'])
        .loc[
    lambda df: ~df["iso_code"].str.contains("_")
].set_index('date'))


In [23]:
plt.figure()
plt.rcParams.update({'font.size': 22}) # must set in top
plt.rcParams.update({'figure.figsize': (24,10)}) # must set in top

<Figure size 1728x720 with 0 Axes>

In [58]:
data[data['iso_code'] == "DEU"].head(3)

,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,
2020-01-27,DEU,Europe,Germany,1.0,1.0,NaN,NaN,NaN,NaN,0.012,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN
2020-01-28,DEU,Europe,Germany,4.0,3.0,NaN,NaN,NaN,NaN,0.048,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN
2020-01-29,DEU,Europe,Germany,4.0,0.0,NaN,NaN,NaN,NaN,0.048,...,28.2,33.1,NaN,8.0,81.33,0.947,NaN,NaN,NaN,NaN


In [59]:
interesting_colums = ["new_cases", "new_cases_smoothed", "new_cases_smoothed_per_million",
                      "new_deaths", "new_deaths_smoothed","new_deaths_smoothed_per_million", "total_cases_per_million", "total_deaths_per_million", "mortality_rate"]
country_codes = ["SVK", "CZE", "DEU", "SWE", "USA", "BRA", "EGY","IND"]

data['mortality_rate'] = data['total_deaths_per_million'] / data['total_cases_per_million']

slovakia = data[data['iso_code'] == "SVK"][interesting_colums]
czech_republic = data[data['iso_code'] == "CZE"][interesting_colums]
germany = data[data['iso_code'] == "DEU"][interesting_colums]
sweden = data[data['iso_code'] == "SWE"][interesting_colums]
usa = data[data['iso_code'] == "USA"][interesting_colums]
brazil = data[data['iso_code'] == "BRA"][interesting_colums]
egypt = data[data['iso_code'] == "EGY"][interesting_colums]
india = data[data['iso_code'] == "IND"][interesting_colums] 

countries_combined = pd.concat(
    [slovakia.add_prefix("SVK_"),
     czech_republic.add_prefix("CZE_"),
     germany.add_prefix("DEU_"),
     sweden.add_prefix("SWE_"),
     usa.add_prefix("USA_"),
     brazil.add_prefix("BRA_"),
     egypt.add_prefix("EGY_"),
     india.add_prefix("IND_")    ],
    axis=1)


In [60]:
countries_combined.tail()

,SVK_new_cases,SVK_new_cases_smoothed,SVK_new_cases_smoothed_per_million,SVK_new_deaths,SVK_new_deaths_smoothed,SVK_new_deaths_smoothed_per_million,SVK_total_cases_per_million,SVK_total_deaths_per_million,SVK_mortality_rate,CZE_new_cases,...,EGY_mortality_rate,IND_new_cases,IND_new_cases_smoothed,IND_new_cases_smoothed_per_million,IND_new_deaths,IND_new_deaths_smoothed,IND_new_deaths_smoothed_per_million,IND_total_cases_per_million,IND_total_deaths_per_million,IND_mortality_rate
date,,,,,,,,,,,,,,,,,,,,,
2022-04-05,6512.0,6021.429,1104.997,22.0,21.571,3.959,453053.712,3571.488,0.007883,6729.0,...,0.048064,1086.0,1101.429,0.790,71.0,55.143,0.040,30881.761,374.253,0.012119
2022-04-06,6057.0,5737.286,1052.854,20.0,21.143,3.880,454165.237,3575.158,0.007872,5305.0,...,0.048064,1033.0,1074.000,0.771,43.0,57.286,0.041,30882.503,374.283,0.012120
2022-04-07,5263.0,5285.000,969.855,18.0,21.143,3.880,455131.054,3578.461,0.007862,4996.0,...,0.048064,1109.0,1041.714,0.748,43.0,56.000,0.040,30883.298,374.314,0.012120
2022-04-08,4222.0,4893.286,897.971,23.0,22.143,4.063,455905.837,3582.682,0.007858,4166.0,...,0.048064,1150.0,1026.000,0.736,83.0,56.000,0.040,30884.124,374.374,0.012122
2022-04-09,3889.0,4515.286,828.604,26.0,22.429,4.116,456619.511,3587.453,0.007857,2258.0,...,0.048064,1054.0,1020.000,0.732,29.0,48.571,0.035,30884.880,374.395,0.012122


In [61]:
countries_combined[[ code + "_new_cases_smoothed_per_million" for code in country_codes]].plot_bokeh( 
    figsize=(1200,600),
     number_format="1",
    title="Počet prípadov na milión obyvateľov vo viacerých krajinách"
    )

Figure(id='8001', ...)

In [28]:
countries_combined[[ code + "_new_deaths_smoothed_per_million" for code in country_codes]].plot_bokeh( 
    figsize=(1200,600),
     number_format="1",
    title="Počet úmrtí na milión obyvateľov vo viacerých krajinách"
    )

Figure(id='5367', ...)

In [29]:
countries_combined[[ code + "_total_deaths_per_million" for code in country_codes]].plot_bokeh( 
    figsize=(1200,600),
    title="Celkový počet úmrtí na milión obyvateľov vo viacerých krajinách",
    number_format="1"
    )



Figure(id='5973', ...)

In [30]:
countries_combined[[ code + "_total_cases_per_million" for code in country_codes]].plot_bokeh( 
    figsize=(1200,600),
    title="Celkový počet prípadov na milión obyvateľov vo viacerých krajinách"
    )


Figure(id='6614', ...)

In [31]:
countries_combined[[ code + "_mortality_rate" for code in country_codes]].plot_bokeh( 
    figsize=(1200,600),
    title="Úmrtnosť - počet celkových úmrtí / počet celkových prípadov",
    number_format="1.0000"
    )

Figure(id='7290', ...)

### TOP 5 VS SLOVENSKO

In [143]:
def get_top7(data, column):
    ranked = (data.groupby('location')
        .max('date')
        .sort_values(column, ascending=False)
        .reset_index())

    ranked.index = ranked.index + 1

    return ranked.head(7).append(ranked[ranked['location'].isin(["Slovakia","Czechia","Germany","Sweden"])])[['location', column]]


In [144]:
get_top7(data, 'total_deaths')

,location,total_deaths
1,United States,985482.0
2,Brazil,661475.0
3,India,521685.0
4,Russia,364011.0
5,Mexico,323691.0
6,Peru,212471.0
7,United Kingdom,169916.0
13,Germany,131715.0
25,Czechia,39880.0
42,Slovakia,19549.0


In [128]:
get_top7(data, 'total_cases')

,location,total_cases
1,United States,80399486.0
2,India,43035271.0
3,Brazil,30146769.0
4,France,26893286.0
5,Germany,22684849.0
6,United Kingdom,21619833.0
7,Russia,17720977.0
40,Slovakia,2488243.0


In [129]:
get_top7(data, 'total_deaths_per_million')

,location,total_deaths_per_million
1,Peru,6369.146
2,Bulgaria,5320.260
3,Bosnia and Herzegovina,4820.652
4,Hungary,4745.716
5,North Macedonia,4438.552
6,Montenegro,4308.567
7,Georgia,4213.808
10,Slovakia,3587.453


In [134]:
get_top7(data, 'total_cases_per_million')

,location,total_cases_per_million
1,Faeroe Islands,706541.904
2,Denmark,530409.740
3,Andorra,521343.434
4,Gibraltar,511590.633
5,Cyprus,509066.356
6,Iceland,497198.963
7,Slovenia,474675.558
10,Slovakia,456619.511
